In [1]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.rodent import RodentMultiClipTracking, RodentTracking
from preprocessing import mjx_preprocess as mjxp

from brax.training.agents.ppo import networks as ppo_networks

import numpy as np
import mediapy as media
import jax.numpy as jp
import mujoco
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from brax import envs
from preprocessing.mjx_preprocess import process_clip_to_train

2024-07-02 21:44:31.935280: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
reference_clip = process_clip_to_train('clips/all_snips.p')

/root/anaconda3/envs/vnl/lib/python3.11/site-packages/jax/_src/interpreters/xla.py:155: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


In [ ]:
envs.register_environment("rodent", RodentMultiClipTracking)

# Define the environment arguments
env_args = {
    "reference_clip": reference_clip,
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R"
    ],
    "appendage_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
        "skull"
    ],
    "walker_body_names": [
        "torso",
        "pelvis",
        "upper_leg_L",
        "lower_leg_L",
        "foot_L",
        "upper_leg_R",
        "lower_leg_R",
        "foot_R",
        "skull",
        "jaw",
        "scapula_L",
        "upper_arm_L",
        "lower_arm_L",
        "finger_L",
        "scapula_R",
        "upper_arm_R",
        "lower_arm_R",
        "finger_R"
    ],
    "joint_names": [
        "vertebra_1_extend",
        "hip_L_supinate",
        "hip_L_abduct",
        "hip_L_extend",
        "knee_L",
        "ankle_L",
        "toe_L",
        "hip_R_supinate",
        "hip_R_abduct",
        "hip_R_extend",
        "knee_R",
        "ankle_R",
        "toe_R",
        "vertebra_C11_extend",
        "vertebra_cervical_1_bend",
        "vertebra_axis_twist",
        "atlas",
        "mandible",
        "scapula_L_supinate",
        "scapula_L_abduct",
        "scapula_L_extend",
        "shoulder_L",
        "shoulder_sup_L",
        "elbow_L",
        "wrist_L",
        "scapula_R_supinate",
        "scapula_R_abduct",
        "scapula_R_extend",
        "shoulder_R",
        "shoulder_sup_R",
        "elbow_R",
        "wrist_R",
        "finger_R"
    ],
    "center_of_mass": "torso",
    "mjcf_path": "./assets/rodent.xml",
    "scale_factor": 0.9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "healthy_z_range": (0.2, 1.0),
    "reset_noise_scale": 0.1,
    "clip_length": 250,
    "sub_clip_length": 10,
    "ref_traj_length": 5,
    "termination_threshold": 5,
    "body_error_multiplier": 1.0,
    # "min_steps": 10,
}

env = envs.get_environment('rodent', **env_args)

In [ ]:
key = random.PRNGKey(100)
reset_fn = jax.jit(jax.vmap(env.reset))
step_fn = jax.jit(jax.vmap(env.step))

In [33]:
n_envs = 1
key_envs = jax.random.split(key, n_envs)
env_state = reset_fn(key_envs)

In [ ]:
# key = random.PRNGKey(100)
# key_envs = jax.random.split(key, 1)
# index = jax.random.choice(
#                 key_envs,
#                 a=jp.arange(5),
#                 p=jp.array([0.1,0.1,0.1,0.1,0.1])
#             )
# int(index)

/var/folders/yt/5gdjwln55x9_stgx851x0yb80000gn/T/ipykernel_11561/2164151359.py:3: FutureWarning: choice accepts a single key, but was given a key array of shape (1, 2) != (). Use jax.vmap for batching. In a future JAX version, this will be an error.
  index = jax.random.choice(
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Intention/.brax-imitation/lib/python3.10/site-packages/jax/_src/random.py:670: FutureWarning: uniform accepts a single key, but was given a key array of shape (1,) != (). Use jax.vmap for batching. In a future JAX version, this will be an error.
  r = p_cuml[-1] * (1 - uniform(key, shape, dtype=p_cuml.dtype))


3